In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import os 

#new libraries 
from matplotlib.patches import Rectangle 
import matplotlib.cm as cm 
import re 

from datetime import datetime,date
from geopy.geocoders import Nominatim 
import folium 
from tqdm import tqdm 

from math import radians, cos, sin, asin,sqrt

from sklearn.cluster import KMeans 
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

import gc 
import lightgbm as lgb
from statistics import variance as var
from statistics import stdev


In [ ]:
PATH='../input/acea-water-prediction/'
Aquifer_Doganella=pd.read_csv(f"{PATH}Aquifer_Doganella.csv")
Aquifer_Auser=pd.read_csv(f"{PATH}Aquifer_Auser.csv")
Water_Spring_Amiata=pd.read_csv(f"{PATH}Water_Spring_Amiata.csv")
Lake_Bilancino=pd.read_csv(f"{PATH}Lake_Bilancino.csv")
Water_Spring_Madonna_di_Canneto=pd.read_csv(f"{PATH}Water_Spring_Madonna_di_Canneto.csv")
Aquifer_Luco=pd.read_csv(f"{PATH}Aquifer_Luco.csv")
Aquifer_Petrignano=pd.read_csv(f"{PATH}Aquifer_Petrignano.csv")
Water_Spring_Lupa=pd.read_csv(f"{PATH}Water_Spring_Lupa.csv")
River_Arno=pd.read_csv(f"{PATH}River_Arno.csv")

print('Shape of Dataset:')
print('-'*30)
print('Shape of Aquifer_Doganella = {}'.format(Aquifer_Doganella.shape))
print('Shape of Aquifer_Auser = {}'.format(Aquifer_Auser.shape))
print('Shape of Water_Spring_Amiata = {}'.format(Water_Spring_Amiata.shape))
print('Shape of Lake_Bilancino = {}'.format(Lake_Bilancino.shape))
print('Shape of Water_Spring_Madonna_di_Canneto = {}'.format(Water_Spring_Madonna_di_Canneto.shape))
print('Shape of Aquifer_Luco = {}'.format(Aquifer_Luco.shape))
print('Shape of Aquifer_Petrignano = {}'.format(Aquifer_Petrignano.shape))
print('Shape of Water_Spring_Lupa = {}'.format(Water_Spring_Lupa.shape))
print('Shape of River_Arno = {}'.format(River_Arno.shape))

In [ ]:
datasets=[]
for dirname,_,filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if '.csv' in filename:
            datasets += list([filename])
datasets

In [ ]:
df=Aquifer_Doganella

In [ ]:
df.info()

In [ ]:
x=['Date','Rainfall_Monteporzio', 'Rainfall_Velletri','Volume_Pozzo_1', 'Volume_Pozzo_2',
       'Volume_Pozzo_3', 'Volume_Pozzo_4', 'Volume_Pozzo_5+6',
       'Volume_Pozzo_7', 'Volume_Pozzo_8', 'Volume_Pozzo_9',
       'Temperature_Monteporzio', 'Temperature_Velletri']
y=['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_2',
       'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4',
       'Depth_to_Groundwater_Pozzo_5', 'Depth_to_Groundwater_Pozzo_6',
       'Depth_to_Groundwater_Pozzo_7', 'Depth_to_Groundwater_Pozzo_8',
       'Depth_to_Groundwater_Pozzo_9']

In [ ]:
for i in df.columns:
    print("Null value = {}".format(df[str(i)].isnull().sum()));

In [ ]:
df.columns

In [ ]:
df['Year'] = df['Date'].str.split("/",expand=True)[2]
df['Month'] = df['Date'].str.split("/",expand=True)[1]

In [ ]:
plt.figure(figsize=(50,20))
plt.subplot(3,7,1)
sns.boxplot(df['Rainfall_Monteporzio'])
plt.subplot(3,7,2)
sns.boxplot(df['Rainfall_Velletri'])
plt.subplot(3,7,3)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_1'])
plt.subplot(3,7,4)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_2'])
plt.subplot(3,7,5)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_3'])
plt.subplot(3,7,6)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_4'])
plt.subplot(3,7,7)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_5'])
plt.subplot(3,7,8)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_6'])
plt.subplot(3,7,9)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_7'])
plt.subplot(3,7,10)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_8'])
plt.subplot(3,7,11)
sns.boxplot(df['Depth_to_Groundwater_Pozzo_9'])
plt.subplot(3,7,12)
sns.boxplot(df['Volume_Pozzo_1'])
plt.subplot(3,7,13)
sns.boxplot(df['Volume_Pozzo_2'])
plt.subplot(3,7,14)
sns.boxplot(df['Volume_Pozzo_3'])
plt.subplot(3,7,15)
sns.boxplot(df['Volume_Pozzo_4'])
plt.subplot(3,7,16)
sns.boxplot(df['Volume_Pozzo_5+6'])
plt.subplot(3,7,17)
sns.boxplot(df['Volume_Pozzo_7'])
plt.subplot(3,7,18)
sns.boxplot(df['Volume_Pozzo_8'])
plt.subplot(3,7,19)
sns.boxplot(df['Volume_Pozzo_9'])
plt.subplot(3,7,20)
sns.boxplot(df['Temperature_Monteporzio'])
plt.subplot(3,7,21)
sns.boxplot(df['Temperature_Velletri'])


In [ ]:
plt.figure(figsize=(50,20))
plt.subplot(3,7,1)
sns.distplot(df['Rainfall_Monteporzio'])
plt.subplot(3,7,2)
sns.distplot(df['Rainfall_Velletri'])
plt.subplot(3,7,3)
sns.distplot(df['Depth_to_Groundwater_Pozzo_1'])
plt.subplot(3,7,4)
sns.distplot(df['Depth_to_Groundwater_Pozzo_2'])
plt.subplot(3,7,5)
sns.distplot(df['Depth_to_Groundwater_Pozzo_3'])
plt.subplot(3,7,6)
sns.distplot(df['Depth_to_Groundwater_Pozzo_4'])
plt.subplot(3,7,7)
sns.distplot(df['Depth_to_Groundwater_Pozzo_5'])
plt.subplot(3,7,8)
sns.distplot(df['Depth_to_Groundwater_Pozzo_6'])
plt.subplot(3,7,9)
sns.distplot(df['Depth_to_Groundwater_Pozzo_7'])
plt.subplot(3,7,10)
sns.distplot(df['Depth_to_Groundwater_Pozzo_8'])
plt.subplot(3,7,11)
sns.distplot(df['Depth_to_Groundwater_Pozzo_9'])
plt.subplot(3,7,12)
sns.distplot(df['Volume_Pozzo_1'])
plt.subplot(3,7,13)
sns.distplot(df['Volume_Pozzo_2'])
plt.subplot(3,7,14)
sns.distplot(df['Volume_Pozzo_3'])
plt.subplot(3,7,15)
sns.distplot(df['Volume_Pozzo_4'])
plt.subplot(3,7,16)
sns.distplot(df['Volume_Pozzo_5+6'])
plt.subplot(3,7,17)
sns.distplot(df['Volume_Pozzo_7'])
plt.subplot(3,7,18)
sns.distplot(df['Volume_Pozzo_8'])
plt.subplot(3,7,19)
sns.distplot(df['Volume_Pozzo_9'])
plt.subplot(3,7,20)
sns.distplot(df['Temperature_Monteporzio'])
plt.subplot(3,7,21)
sns.distplot(df['Temperature_Velletri'])


In [ ]:
df['Rainfall_Monteporzio'] = df['Rainfall_Monteporzio'].fillna(df['Rainfall_Monteporzio'].mean())

uiqr=np.percentile(df['Rainfall_Monteporzio'],75)
filt = (df['Rainfall_Monteporzio'] > uiqr)
df.loc[filt,'Rainfall_Monteporzio'] = uiqr

sns.distplot(df['Rainfall_Monteporzio'])
print(df['Rainfall_Monteporzio'].var())
print(df['Rainfall_Monteporzio'].mean())

In [ ]:
df['Rainfall_Velletri'] = df['Rainfall_Velletri'].fillna(df['Rainfall_Velletri'].mean())

uiqr=np.percentile(df['Rainfall_Velletri'],75)
filt = df['Rainfall_Velletri'] > uiqr 
df.loc[filt,'Rainfall_Velletri'] = uiqr

sns.distplot(df['Rainfall_Velletri'])
print(df['Rainfall_Velletri'].mean())
print(df['Rainfall_Velletri'].var())

In [ ]:
df['Depth_to_Groundwater_Pozzo_1'] = df['Depth_to_Groundwater_Pozzo_1'].fillna(df['Depth_to_Groundwater_Pozzo_1'].mean())
df['Depth_to_Groundwater_Pozzo_2'] = df['Depth_to_Groundwater_Pozzo_2'].fillna(df['Depth_to_Groundwater_Pozzo_2'].mean())
df['Depth_to_Groundwater_Pozzo_3'] = df['Depth_to_Groundwater_Pozzo_3'].fillna(df['Depth_to_Groundwater_Pozzo_3'].mean())
df['Depth_to_Groundwater_Pozzo_4'] = df['Depth_to_Groundwater_Pozzo_4'].fillna(df['Depth_to_Groundwater_Pozzo_4'].mean())
df['Depth_to_Groundwater_Pozzo_5'] = df['Depth_to_Groundwater_Pozzo_5'].fillna(df['Depth_to_Groundwater_Pozzo_5'].mean())
df['Depth_to_Groundwater_Pozzo_6'] = df['Depth_to_Groundwater_Pozzo_6'].fillna(df['Depth_to_Groundwater_Pozzo_6'].mean())
#df['Depth_to_Groundwater_Pozzo_7'] = df['Depth_to_Groundwater_Pozzo_7'].fillna(df['Depth_to_Groundwater_Pozzo_7'].mean())
df['Depth_to_Groundwater_Pozzo_8'] = df['Depth_to_Groundwater_Pozzo_8'].fillna(df['Depth_to_Groundwater_Pozzo_8'].mean())
df['Depth_to_Groundwater_Pozzo_9'] = df['Depth_to_Groundwater_Pozzo_9'].fillna(df['Depth_to_Groundwater_Pozzo_9'].mean())